In [1]:
# !apt-get install -y \
#     libgl1-mesa-dev \
#     libgl1-mesa-glx \
#     libglew-dev \
#     libosmesa6-dev \
#     software-properties-common

# !apt-get install -y patchelf

# !apt-get update --fix-missing
# !pip install mujoco
# !pip install  gymnasium
# !apt-get update --fix-missing
# !pip uninstall stable-baselines3

In [2]:
import os
import time
import wandb
import random
from distutils.util import strtobool

import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Normal
from torch.utils.tensorboard import SummaryWriter

In [3]:
def make_env(env_id, idx, capture_video, run_name, gamma):
    def thunk():
        if capture_video:
            env = gym.make(env_id, render_mode="rgb_array")
        else:
            env = gym.make(env_id)
        env = gym.wrappers.FlattenObservation(env)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        if capture_video and idx == 0:
            env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        env = gym.wrappers.ClipAction(env)
        env = gym.wrappers.NormalizeObservation(env)
        env = gym.wrappers.TransformObservation(env, lambda obs: np.clip(obs, -10, 10))
        env = gym.wrappers.NormalizeReward(env, gamma=gamma)
        env = gym.wrappers.TransformReward(env, lambda reward: np.clip(reward, -10, 10))
        return env

    return thunk

In [4]:
def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer

In [5]:
class Agent(nn.Module):

    def __init__(self, envs):
        super().__init__()
        self.critic = nn.Sequential(
            layer_init(nn.Linear(np.array(envs.single_observation_space.shape).prod(),64)),
            nn.Tanh(),
            layer_init(nn.Linear(64,64)),
            nn.Tanh(),
            layer_init(nn.Linear(64,1), std=1.0),
        )

        self.actor_mean = nn.Sequential(
            layer_init(nn.Linear(np.array(envs.single_observation_space.shape).prod(),64)),
            nn.Tanh(),
            layer_init(nn.Linear(64,64)),
            nn.Tanh(),
            layer_init(nn.Linear(64,np.prod(envs.single_action_space.shape)), std=0.01),
        )

        self.actor_logstd = nn.Parameter(torch.zeros(1,np.prod(envs.single_action_space.shape)))

    def get_value(self, x):
        return self.critic(x)

    def get_action_and_value(self, x, action=None):
        action_value = self.critic(x)
        action_mean = self.actor_mean(x)
        action_logstd = self.actor_logstd.expand_as(action_mean)
        action_std = torch.exp(action_logstd)
        probs = Normal(action_mean,action_std)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action).sum(1), probs.entropy().sum(1),action_value

In [6]:
def train(
    seed,
    env_id,
    total_timesteps,
    learning_rate,
    num_envs,
    num_steps,
    gamma,
    gae_lambda,
    num_minibatches,
    update_epochs,
    clip_coef,
    entropy_coef,
    vf_coef,
    max_grad_norm,
    target_kl):

    batch_size = int(num_envs * num_steps)
    minibatch_size = batch_size // num_minibatches

    run_name = f"{env_id}__{seed}__{int(time.time())}"
    wandb.init(
        project="ppo-mujoco-benchmark",
        config={
            "env":env_id,
            "timesteps":total_timesteps,
            "lr":learning_rate,
            "num_steps":num_steps,
            "gamma":gamma,
            "gae_lambda":gae_lambda,
            "num_minibatches":num_minibatches,
            "update_epochs":update_epochs,
            "clip_coef":clip_coef,
            "entropy_coef":entropy_coef,
            "vf_coef":vf_coef,
            "max_grad_norm":max_grad_norm
        },
        sync_tensorboard=True,
        monitor_gym=True,
        name=run_name
    )
    writer = SummaryWriter(f"runs/{run_name}")

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    envs = gym.vector.SyncVectorEnv(
        [make_env(env_id, i, True, run_name, gamma) for i in range(num_envs)]
    )

    agent = Agent(envs).to(device)
    optimizer = optim.Adam(agent.parameters(),lr=learning_rate, eps=1e-5)

    obs = torch.zeros((num_steps, num_envs) + envs.single_observation_space.shape).to(device)
    actions = torch.zeros((num_steps,num_envs) + envs.single_action_space.shape).to(device)
    logprobs = torch.zeros((num_steps, num_envs)).to(device)
    rewards = torch.zeros((num_steps, num_envs)).to(device)
    dones = torch.zeros((num_steps, num_envs)).to(device)
    values = torch.zeros((num_steps, num_envs)).to(device)

    global_step = 0
    start_time = time.time()
    next_obs, _ = envs.reset(seed=seed)
    next_obs = torch.Tensor(next_obs).to(device)
    next_done = torch.zeros(num_envs).to(device)
    num_updates = total_timesteps // batch_size

    for update in range(1, num_updates + 1):

        frac = 1.0 - (update - 1.0) / num_updates
        lrnow = frac * learning_rate
        optimizer.param_groups[0]["lr"] = lrnow

        for step in range(num_steps):

            global_step += 1 * num_envs
            obs[step] = next_obs
            dones[step] = next_done

            with torch.no_grad():
                action, logprob, _, value = agent.get_action_and_value(next_obs)
                values[step] = value.flatten()

            actions[step] = action
            logprobs[step] = logprob

            next_obs, reward, terminated, truncated, infos = envs.step(action.cpu().numpy())
            done = np.logical_or(terminated, truncated)
            rewards[step] = torch.tensor(reward).to(device).view(-1)
            next_obs, next_done = torch.Tensor(next_obs).to(device), torch.Tensor(next_done).to(device)

            if "final_info" not in infos:
                continue

            for info in infos["final_info"]:
                if info is None:
                    continue

                print(f"global_step={global_step}, episodic_return={info['episode']['r']}")
                writer.add_scalar("charts/episodic_return", info["episode"]["r"], global_step)
                writer.add_scalar("charts/episodic_lenght", info["episode"]["l"], global_step)

        with torch.no_grad():
            next_value = agent.get_value(next_obs).reshape(1, -1)
            advantages = torch.zeros_like(rewards).to(device)
            lastgaelam = 0
            for t in reversed(range(num_steps)):
                if t == num_steps - 1:
                    nextnonterminal = 1.0 - next_done
                    nextvalues = next_value
                else:
                    nextnonterminal = 1.0 - dones[t + 1]
                    nextvalues = values [t + 1]

                delta = rewards[t] + gamma * nextvalues * nextnonterminal - values[t]
                advantages[t] = delta + gamma * gae_lambda * nextnonterminal  * lastgaelam
                lastgaelam = advantages[t]
            returns = advantages + values


        b_obs = obs.reshape((-1, ) + envs.single_observation_space.shape )
        b_logprobs = logprobs.reshape(-1)
        b_actions = actions.reshape((-1, ) + envs.single_action_space.shape)
        b_advantages = advantages.reshape(-1)
        b_returns = returns.reshape(-1)
        b_values = values.reshape(-1)

        b_inds = np.arange(batch_size)
        clipfracs = []

        for epoch in range(update_epochs):
            np.random.shuffle(b_inds)
            for start in range(0, batch_size, minibatch_size):
                end = start + minibatch_size
                mb_inds = b_inds[start:end]

                _, newlogprob, entropy, newvalue = agent.get_action_and_value(b_obs[mb_inds],b_actions[mb_inds])
                logratio = newlogprob - b_logprobs[mb_inds]
                ratio = logratio.exp()

                with torch.no_grad():
                    old_approx_kl = (-logratio).mean()
                    approx_kl = ((ratio -1) - logratio).mean()

                mb_advantages = b_advantages[mb_inds]
                mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + 1e-8)

                pg_loss1 = -mb_advantages * ratio
                pg_loss2 = -mb_advantages * torch.clamp(ratio, 1 - clip_coef, 1 + clip_coef )
                pg_loss = torch.max(pg_loss1, pg_loss2).mean()

                newvalue = newvalue.view(-1)

                v_loss_unclipped = (newvalue - b_returns[mb_inds]) ** 2
                v_clipped = b_values[mb_inds] + torch.clamp(
                    newvalue - b_values[mb_inds],
                    -clip_coef,
                    clip_coef,
                )
                v_loss_clipped = (v_clipped - b_returns[mb_inds]) ** 2
                v_loss_max = torch.max(v_loss_unclipped, v_loss_clipped)
                v_loss = 0.5 * v_loss_max.mean()

                entropy_loss = entropy.mean()
                loss = pg_loss - entropy_coef * entropy_loss + v_loss * vf_coef


                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_norm_(agent.parameters(), max_grad_norm)
                optimizer.step()

            if target_kl is not None:
                if approx_kl > target_kl:
                    break

            y_pred, y_true = b_values.cpu().numpy(), b_returns.cpu().numpy()
            var_y = np.var(y_true)
            explained_var = np.nan if var_y == 0 else 1 - np.var(y_true - y_pred) / var_y

            writer.add_scalar("charts/learning_rate", optimizer.param_groups[0]["lr"], global_step)
            writer.add_scalar("losses/value_loss", v_loss.item(), global_step)
            writer.add_scalar("losses/policy_loss", pg_loss.item(), global_step)
            writer.add_scalar("losses/entropy", entropy_loss.item(), global_step)
            writer.add_scalar("losses/old_approx_kl", old_approx_kl.item(), global_step)
            writer.add_scalar("losses/approx_kl",approx_kl.item(), global_step)
            writer.add_scalar("losses/explained_variance", explained_var, global_step)
            writer.add_scalar("charts/SPS", int(global_step/ (time.time() - start_time)), global_step)

    envs.close()
    writer.close()


In [15]:
seed = 2
total_timesteps = 1000000
learning_rate = 0.00003
num_envs = 1
num_steps = 2048
gamma = 0.99
gae_lambda = 0.95
num_minibatches = 64
update_epochs = 10
clip_coef = 0.5
ent_coef = 0.0
vf_coef = 0.5
max_grad_norm = 0.5
target_kl = None
env = {"hopper":"Hopper-v2","humanoid":"Humanoid-v2","halfCheetah":"HalfCheetah-v2","ant":"Ant-v2"}

In [ ]:
train(seed,
      env["hopper"],
      total_timesteps,
      learning_rate,
      num_envs,
      num_steps,
      gamma,
      gae_lambda,
      num_minibatches,
      update_epochs,
      clip_coef,
      ent_coef,
      vf_coef,
      max_grad_norm,
      target_kl)

charts/SPS,▁▇████████████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
charts/episodic_lenght,▂▁▁▁▁▂▁▁▂▁▃▃▃▄▃▃▃▅▂▄▃▄▅▅▆▅▅▅▅▆▅▆▇▅▄▆█▆█▇
charts/episodic_return,▁▁▁▁▁▂▁▁▁▁▂▂▃▃▃▃▃▄▂▄▃▄▅▄▅▅▄▅▅▆▅▆▆▅▄▅█▆▇▇
charts/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
global_step,▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
losses/approx_kl,▂▃▃▃▄▄▅█▅▅▄▃▄▄▃▅▃▄▅▄▃▃▅▃▆▅▆▄▄▃▃▂▃▃▃▂▂▁▁▁
losses/entropy,███▇▇▆▆▆▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/explained_variance,▃▃▁▄▅▇█▇▇███████████████████████████████
losses/old_approx_kl,▄▆▁▅▅█▃▅▇▇▃▅▆▇▆▁▅▃▄▆▇▁▁▅▆▇▆▅▁▄▅▃▄▅▄▅▄▄▄▃
losses/policy_loss,▇▅▄▃▆▃▅▃▄▆▁▄▄▅▄▅▄▄▂▄▅▅▅▂▂▃▄▆▆▄▂▇▄▅▇▇█▇▇▇
losses/value_loss,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`
/usr/local/lib/python3.9/dist-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment Hopper-v2 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
/usr/local/lib/python3.9/dist-packages/gymnasium/envs/mujoco/mujoco_env.py:211: DeprecationWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.deprecation(


Moviepy - Building video /notebooks/rl-algos/videos/Hopper-v2__2__1700150624/rl-video-episode-0.mp4.
Moviepy - Writing video /notebooks/rl-algos/videos/Hopper-v2__2__1700150624/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/Hopper-v2__2__1700150624/rl-video-episode-0.mp4
global_step=21, episodic_return=[22.78595]
Moviepy - Building video /notebooks/rl-algos/videos/Hopper-v2__2__1700150624/rl-video-episode-1.mp4.
Moviepy - Writing video /notebooks/rl-algos/videos/Hopper-v2__2__1700150624/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/Hopper-v2__2__1700150624/rl-video-episode-1.mp4
global_step=51, episodic_return=[36.201855]
global_step=72, episodic_return=[23.97786]
global_step=82, episodic_return=[4.733545]
global_step=99, episodic_return=[10.860237]
global_step=116, episodic_return=[10.610827]
global_step=125, episodic_return=[6.901435]
global_step=135, episodic_return=[8.571838]
Moviepy - Building video /notebooks/rl-algos/videos/Hopper-v2__2__1700150624/rl-video-episode-8.mp4.
Moviepy - Writing video /notebooks/rl-algos/videos/Hopper-v2__2__1700150624/rl-video-episode-8.mp4



...

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/Hopper-v2__2__1700150624/rl-video-episode-8000.mp4
global_step=190082, episodic_return=[46.46753]
global_step=190112, episodic_return=[53.88542]
global_step=190138, episodic_return=[46.620335]
global_step=190170, episodic_return=[57.297344]
global_step=190193, episodic_return=[40.646786]
global_step=190224, episodic_return=[55.75165]
global_step=190248, episodic_return=[42.257065]
global_step=190274, episodic_return=[45.99583]
global_step=190304, episodic_return=[53.638187]
global_step=190330, episodic_return=[46.24725]
global_step=190357, episodic_return=[48.61723]
global_step=190380, episodic_return=[40.113674]
global_step=190406, episodic_return=[46.529617]
global_step=190431, episodic_return=[44.68707]
global_step=190463, episodic_return=[57.296238]
global_step=190495, episodic_return=[57.710667]
global_step=190521, episodic_return=[46.667427]
global_step=190550, episodic_return=[52.31816]
global_step=190578, episodi

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/Hopper-v2__2__1700150624/rl-video-episode-9000.mp4
global_step=224550, episodic_return=[73.52317]
global_step=224590, episodic_return=[72.30749]
global_step=224627, episodic_return=[68.32905]
global_step=224667, episodic_return=[73.66196]
global_step=224707, episodic_return=[74.13907]
global_step=224748, episodic_return=[73.73415]
global_step=224787, episodic_return=[70.548256]
global_step=224827, episodic_return=[72.609406]
global_step=224863, episodic_return=[65.18446]
global_step=224904, episodic_return=[74.12629]
global_step=224947, episodic_return=[79.16142]
global_step=224985, episodic_return=[69.36971]
global_step=225024, episodic_return=[72.00301]
global_step=225064, episodic_return=[73.675026]
global_step=225102, episodic_return=[68.813]
global_step=225135, episodic_return=[59.713757]
global_step=225172, episodic_return=[67.196526]
global_step=225213, episodic_return=[73.749214]
global_step=225252, episodic_retu

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/Hopper-v2__2__1700150624/rl-video-episode-10000.mp4
global_step=269935, episodic_return=[92.66995]
global_step=269982, episodic_return=[84.59177]
global_step=270031, episodic_return=[89.57412]
global_step=270081, episodic_return=[91.4024]
global_step=270130, episodic_return=[89.92692]
global_step=270179, episodic_return=[90.4864]
global_step=270227, episodic_return=[88.49407]
global_step=270276, episodic_return=[89.7081]
global_step=270323, episodic_return=[87.646225]
global_step=270369, episodic_return=[83.743835]
global_step=270417, episodic_return=[87.32423]
global_step=270461, episodic_return=[81.982544]
global_step=270504, episodic_return=[78.950035]
global_step=270550, episodic_return=[84.06126]
global_step=270600, episodic_return=[90.61787]
global_step=270647, episodic_return=[86.32619]
global_step=270690, episodic_return=[78.80643]
global_step=270738, episodic_return=[86.92409]
global_step=270789, episodic_return

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/Hopper-v2__2__1700150624/rl-video-episode-11000.mp4
global_step=320628, episodic_return=[102.13541]
global_step=320682, episodic_return=[98.4784]
global_step=320738, episodic_return=[102.90048]
global_step=320794, episodic_return=[103.25397]
global_step=320852, episodic_return=[107.07735]
global_step=320919, episodic_return=[124.05827]
global_step=320975, episodic_return=[102.782814]
global_step=321031, episodic_return=[103.658424]
global_step=321084, episodic_return=[97.59829]
global_step=321139, episodic_return=[101.12096]
global_step=321195, episodic_return=[101.96222]
global_step=321254, episodic_return=[107.9639]
global_step=321311, episodic_return=[104.20503]
global_step=321387, episodic_return=[139.71957]
global_step=321464, episodic_return=[141.56984]
global_step=321520, episodic_return=[101.77127]
global_step=321578, episodic_return=[107.68336]
global_step=321638, episodic_return=[109.00622]
global_step=321697, 

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/Hopper-v2__2__1700150624/rl-video-episode-12000.mp4
global_step=441905, episodic_return=[661.0282]
global_step=442113, episodic_return=[645.3384]
global_step=442337, episodic_return=[722.5835]
global_step=442557, episodic_return=[680.65393]
global_step=442746, episodic_return=[593.9192]
global_step=442948, episodic_return=[649.5858]
global_step=443149, episodic_return=[646.6883]
global_step=443354, episodic_return=[656.63275]
global_step=443580, episodic_return=[735.61945]
global_step=443801, episodic_return=[696.93414]
global_step=443990, episodic_return=[603.1377]
global_step=444192, episodic_return=[651.71204]
global_step=444394, episodic_return=[659.04486]
global_step=444527, episodic_return=[354.9696]
global_step=444721, episodic_return=[608.23926]
global_step=444917, episodic_return=[631.1499]
global_step=445135, episodic_return=[671.35333]
global_step=445272, episodic_return=[371.70688]
global_step=445479, episodi